## Tutorial: Data Preprocessing

### University of California, Santa Barbara  
### PSTAT 135/235  
### Last Updated: Oct 22, 2018

---  

### Sources 

Learning PySpark, Chapters 3, 4

https://www.analyticsvidhya.com/blog/2016/10/spark-dataframe-and-operations/

### OBJECTIVES
- Provide additional practice working with RDDs, DataFrames, Schemas, SQL, Temp Tables  
- Introduction to methods for data preprocessing

### PREREQUISITES
- RDDs
- Spark DataFrames
- Schemas

---  

In [30]:
import os
import pyspark.sql.types as typ
import pyspark.sql.functions as F

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master("local") \
    .appName("data preprocessing") \
    .config("spark.executor.memory", '8g') \
    .config('spark.executor.cores', '4') \
    .config('spark.cores.max', '4') \
    .config("spark.driver.memory",'8g') \
    .getOrCreate()

sc = spark.sparkContext

In [6]:
sc.getConf().getAll()

[('spark.master', 'local'),
 ('spark.driver.host', '5b2d7f2b6613'),
 ('spark.executor.id', 'driver'),
 ('spark.driver.port', '33721'),
 ('spark.app.name', 'data preprocessing'),
 ('spark.executor.cores', '4'),
 ('spark.cores.max', '4'),
 ('spark.rdd.compress', 'True'),
 ('spark.driver.memory', '8g'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.app.id', 'local-1540231376869'),
 ('spark.executor.memory', '8g'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true')]

In [7]:
path_to_data = os.path.join('/home/jovyan/UCSB_BigDataAnalytics/data/fraud/ccFraud.csv.gz')

In [8]:
# read in data; notice we can directly read zipped csv format
fraud = sc.textFile(path_to_data)

In [9]:
fraud.take(5)

['"custID","gender","state","cardholder","balance","numTrans","numIntlTrans","creditLine","fraudRisk"',
 '1,1,35,1,3000,4,14,2,0',
 '2,2,2,1,0,9,0,18,0',
 '3,2,2,1,0,27,9,16,0',
 '4,1,15,1,0,12,0,5,0']

In [10]:
type(fraud)

pyspark.rdd.RDD

In [11]:
header = fraud.first()

In [12]:
header

'"custID","gender","state","cardholder","balance","numTrans","numIntlTrans","creditLine","fraudRisk"'

In [13]:
n_rec = fraud.count()

In [14]:
fraud = fraud \
        .filter(lambda row: row != header) \
        .map(lambda row: [int(elem) for elem in row.split(',')])

In [15]:
fraud.take(5)

[[1, 1, 35, 1, 3000, 4, 14, 2, 0],
 [2, 2, 2, 1, 0, 9, 0, 18, 0],
 [3, 2, 2, 1, 0, 27, 9, 16, 0],
 [4, 1, 15, 1, 0, 12, 0, 5, 0],
 [5, 1, 46, 1, 0, 11, 16, 7, 0]]

#### Create a schema to pass to a Spark dF

In [16]:
fields = [
    *[
        typ.StructField(h[1:-1], typ.IntegerType(), True)
        for h in header.split(',')
    ]
]

note: h[1:-1] indexing strips quotes from field names

In [17]:
schema = typ.StructType(fields)

In [18]:
schema

StructType(List(StructField(custID,IntegerType,true),StructField(gender,IntegerType,true),StructField(state,IntegerType,true),StructField(cardholder,IntegerType,true),StructField(balance,IntegerType,true),StructField(numTrans,IntegerType,true),StructField(numIntlTrans,IntegerType,true),StructField(creditLine,IntegerType,true),StructField(fraudRisk,IntegerType,true)))

In [19]:
fraud_df = spark.createDataFrame(fraud, schema)

In [20]:
# cache the table as we will use it many times
fraud_df.cache()

DataFrame[custID: int, gender: int, state: int, cardholder: int, balance: int, numTrans: int, numIntlTrans: int, creditLine: int, fraudRisk: int]

In [21]:
type(fraud_df)

pyspark.sql.dataframe.DataFrame

In [22]:
fraud_df.show(5)

+------+------+-----+----------+-------+--------+------------+----------+---------+
|custID|gender|state|cardholder|balance|numTrans|numIntlTrans|creditLine|fraudRisk|
+------+------+-----+----------+-------+--------+------------+----------+---------+
|     1|     1|   35|         1|   3000|       4|          14|         2|        0|
|     2|     2|    2|         1|      0|       9|           0|        18|        0|
|     3|     2|    2|         1|      0|      27|           9|        16|        0|
|     4|     1|   15|         1|      0|      12|           0|         5|        0|
|     5|     1|   46|         1|      0|      11|          16|         7|        0|
+------+------+-----+----------+-------+--------+------------+----------+---------+
only showing top 5 rows



In [23]:
# Create view so we can query using SQL
fraud_df.createOrReplaceTempView("fraud_v")

In [24]:
# Example of using SQL to count records
spark.sql("select count(*) from fraud_v").show()

+--------+
|count(1)|
+--------+
|10000000|
+--------+



In [25]:
spark.sql("select * from fraud_v where balance > 3000").show(10)

+------+------+-----+----------+-------+--------+------------+----------+---------+
|custID|gender|state|cardholder|balance|numTrans|numIntlTrans|creditLine|fraudRisk|
+------+------+-----+----------+-------+--------+------------+----------+---------+
|     6|     2|   44|         2|   5546|      21|           0|        13|        0|
|     8|     1|   10|         1|   6016|      20|           3|         6|        0|
|    11|     1|   46|         1|   4601|      54|           0|         4|        0|
|    14|     2|   38|         1|   9000|      41|           3|         8|        0|
|    15|     1|   27|         1|   5227|      60|           0|        17|        0|
|    17|     2|   18|         1|  13970|      20|           0|        13|        0|
|    18|     1|   35|         1|   3113|      13|           6|         8|        0|
|    19|     1|    5|         1|   9000|      20|           2|         8|        0|
|    21|     1|   39|         1|   4000|      24|           0|         3|   

In [26]:
fraud_df.printSchema()

root
 |-- custID: integer (nullable = true)
 |-- gender: integer (nullable = true)
 |-- state: integer (nullable = true)
 |-- cardholder: integer (nullable = true)
 |-- balance: integer (nullable = true)
 |-- numTrans: integer (nullable = true)
 |-- numIntlTrans: integer (nullable = true)
 |-- creditLine: integer (nullable = true)
 |-- fraudRisk: integer (nullable = true)



In [27]:
fraud_df.groupby('gender').count().show()

+------+-------+
|gender|  count|
+------+-------+
|     1|6178231|
|     2|3821769|
+------+-------+



### Handle duplicates

In [28]:
# row count
print('rows={}'.format(fraud_df.count()))

rows=10000000


In [29]:
# distinct row count
print('rows={}'.format(fraud_df.distinct().count()))

rows=10000000


In [66]:
# if there were duplicates, you could issue:
fraud_df = fraud_df.dropDuplicates()

### Handle missing

In [35]:
# for each field, compute missing percentage

fraud_df.agg(*[
    (1 - F.count(c) / F.count('*')).alias(c + '_missing')
    for c in fraud_df.columns
]).show()

+--------------+--------------+-------------+------------------+---------------+----------------+--------------------+------------------+-----------------+
|custID_missing|gender_missing|state_missing|cardholder_missing|balance_missing|numTrans_missing|numIntlTrans_missing|creditLine_missing|fraudRisk_missing|
+--------------+--------------+-------------+------------------+---------------+----------------+--------------------+------------------+-----------------+
|           0.0|           0.0|          0.0|               0.0|            0.0|             0.0|                 0.0|               0.0|              0.0|
+--------------+--------------+-------------+------------------+---------------+----------------+--------------------+------------------+-----------------+



notes:  
alias() allow for field renaming  
none of the fields have any missing  
count('*') counts all rows  
the * following agg() treats the list as a separate set of parameters passed to function

#### A more interesting small example with missing values

In [37]:
df_miss = spark.createDataFrame([ (1, 143.5, 5.6, 28, 'M', 100000),
 (2, 167.2, 5.4, 45, 'M', None),
 (3, None , 5.2, None, None, None),
 (4, 144.5, 5.9, 33, 'M', None),
 (5, 133.2, 5.7, 54, 'F', None),
 (6, 124.1, 5.2, None, 'F', None),
 (7, 129.2, 5.3, 42, 'M', 76000),
 ], ['id', 'weight', 'height', 'age', 'gender', 'income'])

In [38]:
df_miss.show()

+---+------+------+----+------+------+
| id|weight|height| age|gender|income|
+---+------+------+----+------+------+
|  1| 143.5|   5.6|  28|     M|100000|
|  2| 167.2|   5.4|  45|     M|  null|
|  3|  null|   5.2|null|  null|  null|
|  4| 144.5|   5.9|  33|     M|  null|
|  5| 133.2|   5.7|  54|     F|  null|
|  6| 124.1|   5.2|null|     F|  null|
|  7| 129.2|   5.3|  42|     M| 76000|
+---+------+------+----+------+------+



In [40]:
df_miss.agg(*[
    (1 - F.count(c) / F.count('*')).alias(c + '_missing')
    for c in df_miss.columns
]).show()

+----------+------------------+--------------+------------------+------------------+------------------+
|id_missing|    weight_missing|height_missing|       age_missing|    gender_missing|    income_missing|
+----------+------------------+--------------+------------------+------------------+------------------+
|       0.0|0.1428571428571429|           0.0|0.2857142857142857|0.1428571428571429|0.7142857142857143|
+----------+------------------+--------------+------------------+------------------+------------------+



#### Impute missing values

In [45]:
# since income is sparse, we drop from the df

df_miss_no_income = df_miss.select([
 c for c in df_miss.columns if c != 'income'
])


In [46]:
df_miss_no_income.show()

+---+------+------+----+------+
| id|weight|height| age|gender|
+---+------+------+----+------+
|  1| 143.5|   5.6|  28|     M|
|  2| 167.2|   5.4|  45|     M|
|  3|  null|   5.2|null|  null|
|  4| 144.5|   5.9|  33|     M|
|  5| 133.2|   5.7|  54|     F|
|  6| 124.1|   5.2|null|     F|
|  7| 129.2|   5.3|  42|     M|
+---+------+------+----+------+



In [63]:
# impute non-categorical fields with column mean
# note: computes to pandas df, then dictionary

means = df_miss_no_income.agg(*[F.mean(c).alias(c) \
                                for c in df_miss_no_income.columns if c != 'gender']) \
                                .toPandas().to_dict('records')[0]

In [64]:
means

{'id': 4.0,
 'weight': 140.28333333333333,
 'height': 5.471428571428571,
 'age': 40.4}

In [65]:
# add impute value for gender

means['gender'] = 'missing'

In [66]:
means

{'id': 4.0,
 'weight': 140.28333333333333,
 'height': 5.471428571428571,
 'age': 40.4,
 'gender': 'missing'}

In [67]:
df_miss_no_income.fillna(means).show()

+---+------------------+------+---+-------+
| id|            weight|height|age| gender|
+---+------------------+------+---+-------+
|  1|             143.5|   5.6| 28|      M|
|  2|             167.2|   5.4| 45|      M|
|  3|140.28333333333333|   5.2| 40|missing|
|  4|             144.5|   5.9| 33|      M|
|  5|             133.2|   5.7| 54|      F|
|  6|             124.1|   5.2| 40|      F|
|  7|             129.2|   5.3| 42|      M|
+---+------------------+------+---+-------+



### Handle outliers

In [68]:
df_outliers = spark.createDataFrame([
 (1, 143.5, 5.3, 28),
 (2, 154.2, 5.5, 45),
 (3, 342.3, 5.1, 99),
 (4, 144.5, 5.5, 33),
 (5, 133.2, 5.4, 54),
 (6, 124.1, 5.1, 21),
 (7, 129.2, 5.3, 42),
 ], ['id', 'weight', 'height', 'age'])

We flag values less than Q1 - 1.5 IQR OR greater than Q3 + 1.5 IQR as outliers  
where IQR = Q3 - Q1

In [80]:
cols = [c for c in df_outliers.columns if c != 'id']   # exclude id from features
bounds = {} # will store lower and upper bounds for each feature

In [81]:
for col in cols:
    quantiles = df_outliers.approxQuantile(col, [0.25, 0.75], 0.05)
    IQR = quantiles[1] - quantiles[0]

    bounds[col] = [
     quantiles[0] - 1.5 * IQR,
     quantiles[1] + 1.5 * IQR
    ]

In [78]:
bounds

{'weight': [91.69999999999999, 191.7],
 'height': [4.499999999999999, 6.1000000000000005],
 'age': [-11.0, 93.0]}

In [87]:
# append outlier indicators to data table

outliers = df_outliers.select(*['id'] + [
 (
 (df_outliers[c] < bounds[c][0]) | (df_outliers[c] > bounds[c][1]))
    .alias(c + '_outlier') for c in cols
])

In [88]:
outliers.show()

+---+--------------+--------------+-----------+
| id|weight_outlier|height_outlier|age_outlier|
+---+--------------+--------------+-----------+
|  1|         false|         false|      false|
|  2|         false|         false|      false|
|  3|          true|         false|       true|
|  4|         false|         false|      false|
|  5|         false|         false|      false|
|  6|         false|         false|      false|
|  7|         false|         false|      false|
+---+--------------+--------------+-----------+



The next step would be to apply a treatment to handle the outliers, such as:  
1. removing the record  
2. clipping the outlier value to something "reasonable"